In [1]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [2]:
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
model = Sequential()
model.add(BatchNormalization(input_shape=(64,64,3)))
model.add(Conv2D(16, (3, 3), padding="same",input_shape=(64,64,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dense(5))
model.add(Activation("softmax"))

model.summary()

# We do not compile the model now!
# model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 64, 64, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 16)        448       
_________________________________________________________________
activation (Activation)      (None, 64, 64, 16)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0

In [5]:
import cv2
import os
def load_img(indir):
    samples = []
    labels = []
    for class_dir in os.listdir(indir):
        the_class = class_dir
        for file in os.listdir(indir+'/'+class_dir):
            if file.endswith('jpg'):
                image = cv2.imread("{}/{}/{}".format(indir,class_dir,file))
                #image = preprocess_input(image)
                image = cv2.resize(image, (64,64))
                samples.append(image)
                labels.append(the_class)
    samples = np.array(samples)
    labels = np.array(labels)
    return samples,labels
samples, labels = load_img('flower_photos')
print('loaded',len(samples),' samples')
print('classes',set(labels))

samples = samples / 255
# one-hot labels
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
print("Labels shape",labels.shape)
labels = labels.astype(float)

loaded 3670  samples
classes {'dandelion', 'tulips', 'sunflowers', 'daisy', 'roses'}
Labels shape (3670, 5)


In [6]:
import sklearn.model_selection
(trainSamples, testSamples, trainLabels, testLabels) = sklearn.model_selection.train_test_split(samples, labels,random_state=42)
testSamples.shape

(918, 64, 64, 3)

In [7]:
from tensorflow.keras.losses import categorical_crossentropy
# opt = tf.keras.optimizers.Adam(learning_rate=0.05
#                                , decay=0.01    ## when decay is used the learning rate decreases!
#                               )
opt = tf.keras.optimizers.Adam(0.0001)
@tf.function
def step(tsamples, tlabels):
    with tf.GradientTape() as tape:
        predictions = model(tsamples)
        loss = categorical_crossentropy(tlabels,predictions)
    #print("loss",loss.numpy().mean())
    # Calculate gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(gradients, model.trainable_variables))
step_no=0

In [8]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, cohen_kappa_score
import time

EPOCHS = 10
for i in range(EPOCHS):
    print('=========== STEP ',step_no)
    step_no+=1
    start = time.time()  
    step(trainSamples,trainLabels)
    end = time.time()
    testResults = model.predict(testSamples)
    print(confusion_matrix(testLabels.argmax(axis=1), testResults.argmax(axis=1)))
    #print(classification_report(testLabels.argmax(axis=1), testResults.argmax(axis=1)))
    accuracy = accuracy_score(testLabels.argmax(axis=1), testResults.argmax(axis=1))
    print(f'Accuracy: {accuracy:.2f} time',(end - start))
    trainResults = model.predict(trainSamples)
    trainAccuracy = accuracy_score(trainLabels.argmax(axis=1), trainResults.argmax(axis=1))
    print(f'Train accuracy: {trainAccuracy:.2f}')
print('Done')

=========== STEP  0
[[  0  97   0   0  83]
 [  0 146   0   0  68]
 [  0  78   0   0  89]
 [  0  96   0   0  74]
 [  0  90   0   0  97]]
Accuracy: 0.26 time 4.460725545883179
Train accuracy: 0.28
=========== STEP  1
[[  0 133   0   1  46]
 [  0 180   0   3  31]
 [  0  92   0   1  74]
 [  0 106   0  12  52]
 [  0 109   0   2  76]]
Accuracy: 0.29 time 3.790700912475586
Train accuracy: 0.32
=========== STEP  2
[[  5  88   0  34  53]
 [  2 113   1  69  29]
 [  0  39   1  61  66]
 [  1  18   0 118  33]
 [  2  23   0  80  82]]
Accuracy: 0.35 time 4.198116302490234
Train accuracy: 0.39
=========== STEP  3
[[ 69  28   0  30  53]
 [ 42  71   1  65  35]
 [ 33  11   1  48  74]
 [  7   9   0 113  41]
 [ 21   9   0  72  85]]
Accuracy: 0.37 time 4.112987279891968
Train accuracy: 0.40
=========== STEP  4
[[ 95  27   0  11  47]
 [ 57  79   0  32  46]
 [ 41  14   1  22  89]
 [ 16  12   0  95  47]
 [ 35   9   0  34 109]]
Accuracy: 0.41 time 4.234241247177124
Train accuracy: 0.43
=========== STEP  5
[[ 46